Housing prices prediction - Barcelona
==================

### Downloading data from website

Download data from idealista and save it as a mongodb database.

In [157]:
from selenium import webdriver

def newBrowser():
    return webdriver.Firefox()
         


In [135]:
from lxml import html
from lxml.cssselect import CSSSelector
import time

#Auxiliary functions to extract information from the browser

def loadPage(page):
    browser.get(page)
    return browser.page_source

def loadPageTree(page_source):
    tree = html.document_fromstring(page_source)
    return tree

def firstHouse(tree):
    element = tree.cssselect('.item-link')
    if element:
        return element[0].get("href")
    return None

def getListOfHouses(tree):
    houses = [elem.get("href") for elem in tree.cssselect('.item-link')]
    return houses
    
def isLastPage(tree):
    return tree.cssselect(".icon-arrow-right-after") == []
    


In [99]:
# Get the details of the house from the page

def getHouseData(tree):
    data = {}
    price = tree.cssselect(".info-data-price")
    data["price"] = price[0].text_content() if price else ""
    features = tree.cssselect(".info-features")
    data["features"] = [ it.text_content() for idx,it in enumerate(features[0].cssselect("span")) if idx%2==0]  if features else []
    descr = tree.cssselect(".adCommentsLanguage")
    data["description"] = descr[0].text_content() if descr else ""
    details = t.cssselect(".details-property_features")
    if details:
        details_house = [ it.text_content() for it in details[0].cssselect("li")] 
        if len(details)>1:
            bdata = [d.cssselect("li") for d in details[1:]] 
            details_building = [ it.text_content() for d in bdata for it in d] 
        else:
            details_building = []
        energy = details[0].cssselect("span") 
        energy_class = energy[0].get("title") if energy else ""
    else:
        details_house = []
        details_building = []
        energy_class = ""
    data["details_house"] = details_house
    data["details_building"] = details_building
    data["energy_class"] = energy_class
    address = t.cssselect("#headerMap")
    address_details = [it.text_content() for it in address[0].cssselect("li")][0:3] if address else []
    data["address"] = address_details
    
    return data


In [236]:
import pymongo

# Connection to Mongo DB
try:
    conn=pymongo.MongoClient()
    print("connected")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e )

db = conn["idealista"]
collection = db["houses2"]
urls = db["urls"]


connected


In [237]:
def saveToMongo(houseUrl,data):
    house_id = houseUrl.split("/")[2]
    data["id"] = house_id
    collection.replace_one( { "id" : house_id },   data, upsert = True )
    
def saveUrl(source,url):
    urls.insert_one( {"source":source , "url":url})
    

In [256]:
browser = newBrowser()

In [224]:
import random

# Method 1. Go to the first house page. From the go to the next indicated at "Siguiente"

def getNextHouseFromSource(page): #needed sometimes: for some reason getNextHouse fails
    text_next = 'class="btn nav next icon-arrow-right-after" href="'
    pstart = page.find(text_next)
    if pstart==-1:
        return None
    pstart += len(text_next)
    pend = page.find('"', pstart)+1
    return page[pstart:pend]

def getNextHouse(page,tree):
    next_link = tree.cssselect(".icon-arrow-right-after")
    next_house = next_link[0].get("href") if next_link else None
    if next_house is None: 
        next_house = getNextHouseFromSource(page)
    return next_house
    
#Not a good solution. Must keep browser open all the time.
#If browser closes you have to restart because you don't have a list of houses
def getHouses(from_start=True):   
    if from_start:
        p = loadPage("https://www.idealista.com/venta-viviendas/barcelona-barcelona/")
        t = loadPageTree(p)
        time.sleep(3)
        next_house = firstHouse(t)
    else:
        p = browser.page_source
        t = loadPageTree(p)
        next_house = getNextHouse(p,t)
        
    while next_house:
        print(next_house,end=" ")
        p = loadPage("https://www.idealista.com"+next_house)
        t = loadPageTree(p)
        data = getHouseData(t)
        saveToMongo(next_house,data)
        time.sleep(random.randint(43,59)+10*random.random())
        next_house = getNextHouse(p,t)
    print("done")
    
    

 
    

In [ ]:
listHouses = []

In [260]:
#Method 2. First retrieve the list of all houses
#Then you can get houses' data one by one by number
#This way you can stop and restart the browser as needed

def pageFormat(start,end,page):
    res = "https://www.idealista.com/venta-viviendas/barcelona-barcelona/"
    res += "con-precio-hasta_{},precio-desde_{}/{}".format(
        end,start, "pagina-{}.htm".format(page) if page>1 else "")
    return res

def saveUrls(tree):
    global listHouses
    l = getListOfHouses(tree)
    listHouses.extend(l)
    for url in l: 
        saveUrl("idealista",url)
    return len(l)
        
def getAllHouseUrls(startPrice=50000,incPrice=30000,finalPrice=3000000,startPage=1,loopLimit=100):
    pageNumber = startPage
    price = startPrice
    incr = incPrice
    zeros = 0 # sanity check. If something goes wrong, there will be many 0 urls in sequence
    
    while (price<finalPrice) and (zeros<3) and (loopLimit>0):
        pageUrl = pageFormat(price,price+incr-1,pageNumber)
        print(pageUrl,end="")
        p = loadPage(pageUrl)
        t = loadPageTree(p)
        l = saveUrls(t)
        print(" (",l,")",loopLimit)
        zeros = 0 if l>0 else zeros+1
        time.sleep(random.randint(41,59)+10*random.random())
        if isLastPage(t):
            price += incr
            pageNumber = 1
        else:
            pageNumber += 1
        loopLimit -= 1
    print("done")        
       

In [255]:
len(listHouses)
len(set(listHouses))

11536

In [261]:
getAllHouseUrls(845000,25000,3000000,3,100)

https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_869999,precio-desde_845000/pagina-3.htm ( 30 ) 100
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_869999,precio-desde_845000/pagina-4.htm ( 30 ) 99
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_869999,precio-desde_845000/pagina-5.htm ( 30 ) 98
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_869999,precio-desde_845000/pagina-6.htm ( 11 ) 97
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_894999,precio-desde_870000/ ( 30 ) 96
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_894999,precio-desde_870000/pagina-2.htm ( 30 ) 95
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_894999,precio-desde_870000/pagina-3.htm ( 30 ) 94
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_894999,precio-desde_870000/pa

https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_1369999,precio-desde_1345000/ ( 30 ) 36
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_1369999,precio-desde_1345000/pagina-2.htm ( 30 ) 35
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_1369999,precio-desde_1345000/pagina-3.htm ( 21 ) 34
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_1394999,precio-desde_1370000/ ( 30 ) 33
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_1394999,precio-desde_1370000/pagina-2.htm ( 30 ) 32
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_1394999,precio-desde_1370000/pagina-3.htm ( 14 ) 31
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_1419999,precio-desde_1395000/ ( 30 ) 30
https://www.idealista.com/venta-viviendas/barcelona-barcelona/con-precio-hasta_1419999,precio-desde_1395000/pagina-2.ht

In [251]:

urls.count()

/home/egon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


14643

In [182]:
print(t.cssselect(".icon-arrow-right-after")[0].get("href"))

/inmueble/87916907/


In [178]:
p = browser.page_source


'/inmueble/87916907/"'